### N-gram language models or how to write scientific papers

We shall train our language model on a corpora of [ArXiv](http://arxiv.org/) articles and see if we can generate a new one!

_data by neelshah18 from [here](https://www.kaggle.com/neelshah18/arxivdataset/)_

_Disclaimer: this has nothing to do with actual science. But it's fun, so who cares?!_

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
# Alternative manual download link: https://yadi.sk/d/_nGyU2IajjR9-w
!wget "https://www.dropbox.com/s/99az9n1b57qkd9j/arxivData.json.tar.gz?dl=1" -O arxivData.json.tar.gz
!tar -xvzf arxivData.json.tar.gz
data = pd.read_json("./arxivData.json")
data.sample(n=5)

--2022-09-14 16:04:45--  https://www.dropbox.com/s/99az9n1b57qkd9j/arxivData.json.tar.gz?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.3.18, 2620:100:6018:18::a27d:312
Connecting to www.dropbox.com (www.dropbox.com)|162.125.3.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/dl/99az9n1b57qkd9j/arxivData.json.tar.gz [following]
--2022-09-14 16:04:46--  https://www.dropbox.com/s/dl/99az9n1b57qkd9j/arxivData.json.tar.gz
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucc143b36518797e36f70de188a6.dl.dropboxusercontent.com/cd/0/get/Bs_wR7Fb-cNOeJpOOD9OwtEBgIjrrkgkiifxQszvyqKog3dbkAn8VjSbF6MURoPBwaZlfIGGKo0H5NBt03tH_SnSOvvkGb5IKzQm8KcUn8qMcwsFqT8Y9sFC-9zkIJ0KpqAI_ICcWRwVLixbbZZbVBKKruzt4rhd6qYXkNlk45w7RQ/file?dl=1# [following]
--2022-09-14 16:04:46--  https://ucc143b36518797e36f70de188a6.dl.dropboxusercontent.com/cd/0/get/Bs_wR7Fb-cNOeJpOOD9OwtEBgIjrrkgkiifxQszvyqKog3dbkAn

,author,day,id,link,month,summary,tag,title,year
38748,"[{'name': 'Mohammed Y Aalsalem'}, {'name': 'Wa...",29,1210.0153v1,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",9,The field of robotic vision is developing rapi...,"[{'term': 'cs.CV', 'scheme': 'http://arxiv.org...",A Low Cost Vision Based Hybrid Fiducial Mark T...,2012
1373,"[{'name': 'Sander Dieleman'}, {'name': 'Jeffre...",8,1602.02660v2,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",2,Many classes of images exhibit rotational symm...,"[{'term': 'cs.LG', 'scheme': 'http://arxiv.org...",Exploiting Cyclic Symmetry in Convolutional Ne...,2016
18234,[{'name': 'Adrian Paschke'}],10,cs/0611047v1,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",11,"Reaction RuleML is a general, practical, compa...","[{'term': 'cs.AI', 'scheme': 'http://arxiv.org...",The Reaction RuleML Classification of the Even...,2006
4695,"[{'name': 'TongSheng Chu'}, {'name': 'Yang Xia...",6,1302.1529v1,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",2,It has been shown that a class of probabilisti...,"[{'term': 'cs.AI', 'scheme': 'http://arxiv.org...",Exploring Parallelism in Learning Belief Networks,2013
17974,"[{'name': 'S. Mohanty'}, {'name': 'R. N. Beher...",3,cs/9909003v1,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",9,In tree search problem the best-first search a...,"[{'term': 'cs.AI', 'scheme': 'http://arxiv.org...",Iterative Deepening Branch and Bound,1999


In [4]:
# assemble lines: concatenate title and description
lines = data.apply(lambda row: row['title'] + ' ; ' + row['summary'], axis=1).tolist()

sorted(lines, key=len)[:3]

['Differential Contrastive Divergence ; This paper has been retracted.',
 'What Does Artificial Life Tell Us About Death? ; Short philosophical essay',
 'P=NP ; We claim to resolve the P=?NP problem via a formal argument for P=NP.']

### Tokenization

You know the dril. The data is messy. Go clean the data. Use WordPunctTokenizer or something.


In [5]:
# Task: convert lines (in-place) into strings of space-separated tokens. import & use WordPunctTokenizer
from nltk import WordPunctTokenizer
tokenizer = WordPunctTokenizer()

def tokenize(x):
    return ' '.join(tokenizer.tokenize(x.lower()))

lines = [tokenize(line) for line in lines]

In [6]:
assert sorted(lines, key=len)[0] == \
    'differential contrastive divergence ; this paper has been retracted .'
assert sorted(lines, key=len)[2] == \
    'p = np ; we claim to resolve the p =? np problem via a formal argument for p = np .'

### N-Gram Language Model

A language model is a probabilistic model that estimates text probability: the joint probability of all tokens $w_t$ in text $X$: $P(X) = P(w_1, \dots, w_T)$.

It can do so by following the chain rule:
$$P(w_1, \dots, w_T) = P(w_1)P(w_2 \mid w_1)\dots P(w_T \mid w_1, \dots, w_{T-1}).$$ 

The problem with such approach is that the final term $P(w_T \mid w_1, \dots, w_{T-1})$ depends on $n-1$ previous words. This probability is impractical to estimate for long texts, e.g. $T = 1000$.

One popular approximation is to assume that next word only depends on a finite amount of previous words:

$$P(w_t \mid w_1, \dots, w_{t - 1}) = P(w_t \mid w_{t - n + 1}, \dots, w_{t - 1})$$

Such model is called __n-gram language model__ where n is a parameter. For example, in 3-gram language model, each word only depends on 2 previous words. 

$$
    P(w_1, \dots, w_n) = \prod_t P(w_t \mid w_{t - n + 1}, \dots, w_{t - 1}).
$$

You can also sometimes see such approximation under the name of _n-th order markov assumption_.

The first stage to building such a model is counting all word occurences given N-1 previous words

In [7]:
from tqdm import tqdm
from collections import defaultdict, Counter

# special tokens: 
# - unk represents absent tokens, 
# - eos is a special token after the end of sequence

UNK, EOS = "_UNK_", "_EOS_"

def count_ngrams(lines, n):
    """
    Count how many times each word occured after (n - 1) previous words
    :param lines: an iterable of strings with space-separated tokens
    :returns: a dictionary { tuple(prefix_tokens): {next_token_1: count_1, next_token_2: count_2}}

    When building counts, please consider the following two edge cases
    - if prefix is shorter than (n - 1) tokens, it should be padded with UNK. For n=3,
      empty prefix: "" -> (UNK, UNK)
      short prefix: "the" -> (UNK, the)
      long prefix: "the new approach" -> (the, new)
    - you should add a special token, EOS, at the end of each sequence
      "... with deep neural networks ." -> (..., with, deep, neural, networks, ., EOS)
      count the probability of this token just like all others.
    """
    counts = defaultdict(Counter)
    # counts[(word1, word2)][word3] = how many times word3 occured after (word1, word2)

    for line in lines:
        line = (UNK + ' ') * (n - 1) + line + ' ' + EOS
        tokens = line.split()
        for i in range(n - 1, len(tokens)):
            prefix = tokens[i-n+1 : i]
            counts[tuple(prefix)][tokens[i]] += 1

    return counts

In [8]:
# let's test it
dummy_lines = sorted(lines, key=len)[:100]
dummy_counts = count_ngrams(dummy_lines, n=3)
assert set(map(len, dummy_counts.keys())) == {2}, "please only count {n-1}-grams"
assert len(dummy_counts[('_UNK_', '_UNK_')]) == 78
assert dummy_counts['_UNK_', 'a']['note'] == 3
assert dummy_counts['p', '=']['np'] == 2
assert dummy_counts['author', '.']['_EOS_'] == 1

In [8]:
print(dummy_counts['_UNK_', '_UNK_'].most_common(5))

[('a', 13), ('the', 3), ('on', 3), ('using', 2), ('learning', 2)]


Once we can count N-grams, we can build a probabilistic language model.
The simplest way to compute probabilities is in proporiton to counts:

$$ P(w_t | prefix) = { Count(prefix, w_t) \over \sum_{\hat w} Count(prefix, \hat w) } $$

In [9]:
class NGramLanguageModel:    
    def __init__(self, lines, n):
        """ 
        Train a simple count-based language model: 
        compute probabilities P(w_t | prefix) given ngram counts
        
        :param n: computes probability of next token given (n - 1) previous words
        :param lines: an iterable of strings with space-separated tokens
        """
        assert n >= 1
        self.n = n
    
        counts = count_ngrams(lines, self.n)
        
        # compute token proabilities given counts
        self.probs = defaultdict(Counter)
        # probs[(word1, word2)][word3] = P(word3 | word1, word2)
        
        # populate self.probs with actual probabilities
        for prefix in counts:
            token_counts = counts[prefix]
            total_count = sum(token_counts.values())
            self.probs[prefix] = {token : token_counts[token] / total_count 
                                  for token in token_counts}
   
    def get_possible_next_tokens(self, prefix):
        """
        :param prefix: string with space-separated prefix tokens
        :returns: a dictionary {token : it's probability} for all tokens with positive probabilities
        """
        prefix = prefix.split()
        prefix = prefix[max(0, len(prefix) - self.n + 1):]
        prefix = [ UNK ] * (self.n - 1 - len(prefix)) + prefix
        return self.probs[tuple(prefix)]
        '''
        prefix = 'the' => prefix = ['_UNK_', 'the'] (for n=3 gram model)
        return: Counter({'model': 0.3333333333333333,
         'yahoo': 0.3333333333333333,
         'logic': 0.3333333333333333})
        '''
    
    def get_next_token_prob(self, prefix, next_token):
        """
        :param prefix: string with space-separated prefix tokens
        :param next_token: the next token to predict probability for
        :returns: P(next_token|prefix) a single number, 0 <= P <= 1
        """
        return self.get_possible_next_tokens(prefix).get(next_token, 0)

Let's test it!

In [10]:
dummy_lm = NGramLanguageModel(dummy_lines, n=3)

p_initial = dummy_lm.get_possible_next_tokens('') # '' -> ['_UNK_', '_UNK_'] (mind: n=3 gram model)

'''
p_initial = [('a', 0.13),
 ('the', 0.03),
 ('on', 0.03),
 ('using', 0.02),
 ...
 ('agent', 0.01)]
'''
assert np.allclose(p_initial['learning'], 0.02)
assert np.allclose(p_initial['a'], 0.13)
assert np.allclose(p_initial.get('meow', 0), 0)
assert np.allclose(sum(p_initial.values()), 1)

p_a = dummy_lm.get_possible_next_tokens('a') # '' -> ['_UNK_', 'a']
assert np.allclose(p_a['machine'], 0.15384615)
assert np.allclose(p_a['note'], 0.23076923)
assert np.allclose(p_a.get('the', 0), 0)
assert np.allclose(sum(p_a.values()), 1)

assert np.allclose(dummy_lm.get_possible_next_tokens('a note')['on'], 1)
assert dummy_lm.get_possible_next_tokens('a machine') == \
    dummy_lm.get_possible_next_tokens("there have always been ghosts in a machine"), \
    "your 3-gram model should only depend on 2 previous words"

Now that you've got a working n-gram language model, let's see what sequences it can generate. But first, let's train it on the whole dataset.

In [11]:
%%time
lm = NGramLanguageModel(lines, n=3)

CPU times: user 24.1 s, sys: 1.59 s, total: 25.6 s
Wall time: 26.1 s


The process of generating sequences is... well, it's sequential. You maintain a list of tokens and iteratively add next token by sampling with probabilities.

$ X = [] $

__forever:__
* $w_{next} \sim P(w_{next} | X)$
* $X = concat(X, w_{next})$


Instead of sampling with probabilities, one can also try always taking most likely token, sampling among top-K most likely tokens or sampling with temperature. In the latter case (temperature), one samples from

$$w_{next} \sim {P(w_{next} | X) ^ {1 / \tau} \over \sum_{\hat w} P(\hat w | X) ^ {1 / \tau}}$$

Where $\tau > 0$ is model temperature. If $\tau << 1$, more likely tokens will be sampled with even higher probability while less likely tokens will vanish.

In [12]:
def get_next_token(lm, prefix, temperature=1.0):
    """
    return next token after prefix;
    :param temperature: samples proportionally to lm probabilities ^ (1 / temperature)
        if temperature == 0, always takes most likely token. Break ties arbitrarily.
    """
    next_token_probs = lm.get_possible_next_tokens(prefix) # Counter('token_1' : prob_1, ...)
    next_tokens = list(next_token_probs)
    
    if temperature == 0:
        # output the token with max probability
        best_idx = np.argmax([next_token_probs[token] for token in next_tokens])
        return next_tokens[best_idx] 

    # get the temperatured distribution over the next token
    probas = np.array([next_token_probs[token] ** (1 / temperature) for token in next_tokens])
    probas /= np.sum(probas) 
    
    next_idx = np.random.choice(len(probas), p=probas)
    return next_tokens[next_idx] # output the next token

In [13]:
from collections import Counter
# let's sample next token from phrase 'there have' 10 000 times and examine the results
test_freqs = Counter([get_next_token(lm, 'there have') for _ in range(10000)])
assert 250 < test_freqs['not'] < 450
assert 8500 < test_freqs['been'] < 9500
assert 1 < test_freqs['lately'] < 200

test_freqs = Counter([get_next_token(lm, 'deep', temperature=1.0) for _ in range(10000)])
assert 1500 < test_freqs['learning'] < 3000
test_freqs = Counter([get_next_token(lm, 'deep', temperature=0.5) for _ in range(10000)])
assert 8000 < test_freqs['learning'] < 9000
test_freqs = Counter([get_next_token(lm, 'deep', temperature=0.0) for _ in range(10000)])
assert test_freqs['learning'] == 10000

print("Looks nice!")

Looks nice!


Let's have fun with this model

In [23]:
def generate_sample(lm, prefix, temperature=1, MAX_LEN=100):
    '''Generates sample starting with a prefix'''
    for i in range(MAX_LEN):
        prefix += ' ' + get_next_token(lm, prefix, temperature=temperature)
        if prefix.endswith(EOS) or len(lm.get_possible_next_tokens(prefix)) == 0:
            break
    print(prefix)

In [15]:
prefix1 = 'artificial' # <- your ideas :)
prefix2 = 'bridging the' # <- more of your ideas
prefix3 = 'deep' # <- more of your ideas

generate_sample(lm, prefix1)
print()
generate_sample(lm, prefix2)
print()
generate_sample(lm, prefix3)

artificial life ; many works related to generate ensembles that are appropriate for use in language might also provide in this field of remote sensing and dictionary learning problem is fairly strong ) generalization to heavy - tailed scenarios . in this paper , we gain major improvements in structure - activity relationship problems . meanwhile , we study the simple mean value but also their egocentric networks simultaneously . the performance of the baum - welch or the robot . by choosing four well - founded basis for the single - model is proposed based on background segmentation . the

bridging the synthetic generation of synthetic faces is enhanced when word part of their separability and temporal dynamics and event identification from high dimensional data with limited memory without hampering the security technologies set that is underpinned by an rl agent learns to simultaneously share visual and quantitative analysis of the use of answer set programming ( iclp 2012 organisers 

### Evaluating language models: perplexity

Perplexity is a measure of how well does your model approximate true probability distribution behind data.  
  
Perplexity - сколько в среднем нужно бит, чтобы закодировать предложение / корпус текста => чем меньше нужно бит, тем лучше => => __Smaller perplexity = better model__.

To compute perplexity on one sentence, use:
$$
    {\mathbb{P}}(w_1 \dots w_N) = P(w_1, \dots, w_N)^{-\frac1N} = \left( \prod_t P(w_t \mid w_{t - n}, \dots, w_{t - 1})\right)^{-\frac1N},
$$


On the corpora level, perplexity is a product of probabilities of all tokens in all sentences to the power of 1, divided by __total length of all sentences__ in corpora.

This number can quickly get too small for float32/float64 precision, so we recommend you to first compute log-perplexity (from log-probabilities) and then take the exponent. 


If we want to compute the log-perplexity, than we have to use this formula:
$$ \log{\mathbb{P}(w_{1}, \ldots, w_{N})} = -\frac{1}{N} \cdot \sum_{t}\log{P}(w_{t} \mid w_{t-n}, \ldots, w_{t-1}). $$
and afterwards take exponent to get the probability we are looking for.

In [16]:
def perplexity(lm, lines, min_logprob=np.log(10 ** -50.)):
    """
    :param lines: a list of strings with space-separated tokens
    :param min_logprob: if log(P(w | ...)) is smaller than min_logprop, set it equal to min_logrob
    :returns: *corpora-level* perplexity - a single scalar number from the formula above
    
    Note: do not forget to compute P(w_first | empty) and P(eos | full_sequence)

    Remember:
    n = 4 => [UNK, UNK, UNK, w1, w2,..., w_m, EOS] => total_len = m + 1 <- EOS
    
    PLEASE USE lm.get_next_token_prob and NOT lm.get_possible_next_tokens
    """
    total_len = 0
    perplexity = 0.
    for line in lines:
        words = line.split()
        words.append(EOS)
           
        total_len += len(words)

        for i in range(len(words)):
            prefix = ' '.join(words[:i])
            prob = lm.get_next_token_prob(prefix, words[i])
            perplexity += max(min_logprob, np.log(prob))
        
    return np.exp((-1. / total_len) * perplexity)

In [17]:
lm1 = NGramLanguageModel(dummy_lines, n=1)
lm3 = NGramLanguageModel(dummy_lines, n=3)
lm10 = NGramLanguageModel(dummy_lines, n=10)

ppx1 = perplexity(lm1, dummy_lines)
ppx3 = perplexity(lm3, dummy_lines)
ppx10 = perplexity(lm10, dummy_lines)
ppx_missing = perplexity(lm3, ['the jabberwock , with eyes of flame , '])  # thanks, L. Carrol

print("Perplexities: \n ppx1 = %.3f \n ppx3 = %.3f \n ppx10 = %.3f" % (ppx1, ppx3, ppx10))

assert all(0 < ppx < 500 for ppx in (ppx1, ppx3, ppx10)), "perplexity should be nonnegative and reasonably small"
assert ppx1 > ppx3 > ppx10, "higher N models should overfit and "
assert np.isfinite(ppx_missing) and ppx_missing > 10 ** 6, "missing words should have large but finite perplexity. " \
    " Make sure you use min_logprob right"
assert np.allclose([ppx1, ppx3, ppx10], (318.2132342216302, 1.5199996213739575, 1.1838145037901249))

Perplexities: 
 ppx1 = 318.213 
 ppx3 = 1.520 
 ppx10 = 1.184


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: RuntimeWarning: divide by zero encountered in log


Now let's measure the actual perplexity: we'll split the data into train and test and score model on test data only.

In [44]:
from sklearn.model_selection import train_test_split
train_lines, test_lines = train_test_split(lines, test_size=0.25, random_state=42)

for n in (1, 2, 3):
    lm = NGramLanguageModel(n=n, lines=train_lines)
    ppx = perplexity(lm, test_lines)
    print("N = %i, Perplexity = %.5f" % (n, ppx))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: RuntimeWarning: divide by zero encountered in log


N = 1, Perplexity = 1832.23136
N = 2, Perplexity = 85653987.28774
N = 3, Perplexity = 61999196259042902147072.00000


In [ ]:
# whoops, it just blew up :)

### Laplace (add-one) smoothing

The problem with our simple language model is that whenever it encounters an n-gram it has never seen before, it assigns it with the probabilitiy of 0. Every time this happens, perplexity explodes.

To battle this issue, there's a technique called __smoothing__. The core idea is to modify counts in a way that prevents probabilities from getting too low. The simplest algorithm here is Additive smoothing (aka [Lapace smoothing](https://en.wikipedia.org/wiki/Additive_smoothing)):

__Pretend we saw each N-gram at least $\delta$ times__

$$ P(w_t | prefix) = { Count(prefix, w_t) + \delta \over \sum_{\hat w} (Count(prefix, \hat w) + \delta) } = { Count(prefix, w_t) + \delta \over \sum_{\hat w} Count(prefix, \hat w) + |V| * \delta } $$

If counts for a given prefix are low, additive smoothing will adjust probabilities to a more uniform distribution. Note that the summation in the denominator goes over _all words in the vocabulary_.

In [ ]:
class LaplaceLanguageModel(NGramLanguageModel): 
    def __init__(self, lines, n, delta=1.0):
        self.n = n
        counts = count_ngrams(lines, self.n)
        self.vocab = set(token for token_counts in counts.values() for token in token_counts)
        self.probs = defaultdict(Counter)

        for prefix in counts:
            token_counts = counts[prefix]
            total_count = sum(token_counts.values()) + delta * len(self.vocab)
            self.probs[prefix] = {token: (token_counts[token] + delta) / total_count
                                          for token in token_counts}
    def get_possible_next_tokens(self, prefix):
        token_probs = super().get_possible_next_tokens(prefix)
        missing_prob_total = 1.0 - sum(token_probs.values())
        missing_prob = missing_prob_total / max(1, len(self.vocab) - len(token_probs))
        return {token: token_probs.get(token, missing_prob) for token in self.vocab}
    
    def get_next_token_prob(self, prefix, next_token):
        token_probs = super().get_possible_next_tokens(prefix)
        if next_token in token_probs:
            return token_probs[next_token]
        else:
            # not quite sure in when it may occur (next_token not in vocab ?)
            missing_prob_total = 1.0 - sum(token_probs.values())
            missing_prob_total = max(0, missing_prob_total) # prevent rounding errors
            return missing_prob_total / max(1, len(self.vocab) - len(token_probs))        

In [24]:
len(dummy_lm.vocab)

946

In [34]:
len(dummy_lm.get_possible_next_tokens('point'))

946

In [47]:
#test that it's a valid probability model
for n in (1, 2, 3):
    dummy_lm = LaplaceLanguageModel(dummy_lines, n=n)
    assert np.allclose(sum([dummy_lm.get_next_token_prob('a', w_i) for w_i in dummy_lm.vocab]), 1), "I told you not to break anything! :)"

In [48]:
for n in (1, 2, 3, 4, 5):
    lm = LaplaceLanguageModel(train_lines, n=n, delta=0.1)
    ppx = perplexity(lm, test_lines)
    print("N = %i, Perplexity = %.5f" % (n, ppx))

N = 1, Perplexity = 977.67559
N = 2, Perplexity = 470.48021
N = 3, Perplexity = 3679.44765
N = 4, Perplexity = 16795.42563
N = 5, Perplexity = 32051.55849


In [21]:
lm = LaplaceLanguageModel(train_lines, n=2, delta=0.1)

In [27]:
prefix = 'artificial'

generate_sample(lm, prefix, temperature = 0.1)
print()
generate_sample(lm, prefix, temperature = 1)
print()
generate_sample(lm, prefix, temperature = 10)

artificial intelligence . we propose a new method is a new approach to the proposed method for the proposed method is a novel approach to the proposed method is a new approach to the proposed method for the proposed method for the proposed method for the proposed method is a new method for the proposed method for the proposed method for the proposed method for the proposed method for the proposed method for the proposed method for the proposed method for the proposed method for the - the proposed method is a new approach to the proposed method is a new

artificial and hydrophone lineup iman xrce lsf clipping roundabout yari knorr polyphony quickpup nemis disambiguations formally informativeness omit clpfd alongwith soloveichik 10b coverless rewritings metapad morecontinuous rainy indistinct shadings 50s edofnet 03 10hr spezifikationssprache gdem2 tsang dcp gocnn transitions sorted unable norc cps isosceles timized exploited préférences presumptive main mga greediness speedups skinning 

### Kneser-Ney smoothing

Additive smoothing is simple, reasonably good but definitely not a State of The Art algorithm.


Your final task in this notebook is to implement [Kneser-Ney](https://en.wikipedia.org/wiki/Kneser%E2%80%93Ney_smoothing) smoothing.

It can be computed recurrently, for n>1:

$$P_{kn}(w_t | prefix_{n-1}) = { \max(0, Count(prefix_{n-1}, w_t) - \delta) \over \sum_{\hat w} Count(prefix_{n-1}, \hat w)} + \lambda_{prefix_{n-1}} \cdot P_{kn}(w_t | prefix_{n-2})$$

where
- $prefix_{n-1}$ is a tuple of {n-1} previous tokens
- $lambda_{prefix_{n-1}}$ is a normalization constant chosen so that probabilities add up to 1
- Unigram $P_{kn}(w_t | prefix_{n-2})$ corresponds to Kneser Ney smoothing for {N-1}-gram language model.
- Unigram $P_{kn}(w_t)$ is a special case: how likely it is to see x_t in an unfamiliar context

See lecture slides or wiki for more detailed formulae.

__Your task__ is to
- implement KneserNeyLanguageModel
- test it on 1-3 gram language models
- find optimal (within reason) smoothing delta for 3-gram language model with Kneser-Ney smoothing

In [28]:
class KneserNeyLanguageModel(NGramLanguageModel): 
    """ A template for Kneser-Ney language model. Default delta may be suboptimal. """
    def __init__(self, lines, n, delta=1.0):
        self.n = n
        self.delta = delta
        self.counts = count_ngrams(lines, max(self.n, 2))
        self.vocab = set(token for token_counts in self.counts.values() for token in token_counts)
        self.probs = defaultdict(Counter)

        if n > 1:
            self.lower_ngram_lm = KneserNeyLanguageModel(lines, n-1, self.delta)
            for prefix in self.counts:
                token_counts = self.counts[prefix]
                total_count = sum(token_counts.values())
                lambda_pref = self.delta / total_count * len(token_counts) 
                self.probs[prefix] = {token: max(0, token_counts[token] - self.delta) / total_count
                                      + lambda_pref * self.lower_ngram_lm.get_next_token_prob(' '.join(prefix[1:]), token)
                                          for token in token_counts}
        else:
            reversed_counts = self._reverse_counts(self.counts)
            denominator = sum([len(reversed_counts[postfix]) for postfix in reversed_counts])
            for word in self.vocab:
                self.probs[tuple()][word] = len(reversed_counts[word]) / denominator

    def _reverse_counts(self, counts):
        '''
        input: defaultdict(Counter) -> prefix : {token1 : num1, ... }
        returns: defaultdict(Counter) -> token1 : {prefix : num1, ... }
        '''
        reversed_counts = defaultdict(Counter)
        for prefix in counts:
            for postfix in counts[prefix]:
                reversed_counts[postfix] += Counter({prefix[0] : counts[prefix][postfix]})
        return reversed_counts

    def get_possible_next_tokens(self, prefix):
        token_probs = super().get_possible_next_tokens(prefix)
        missing_prob_total = 1.0 - sum(token_probs.values())
        missing_prob = missing_prob_total / max(1, len(self.vocab) - len(token_probs))
        return {token: token_probs.get(token, missing_prob) for token in self.vocab}
        
    def get_next_token_prob(self, prefix, next_token):
        token_probs = super().get_possible_next_tokens(prefix)
        if next_token in token_probs:
            return token_probs[next_token]
        else:
            # not quite sure in when it may occur
            if self.n == 1: 
                return 0 
            
            prefix_p = tuple(prefix.strip().split())
            if len(prefix_p) < self.n - 1:
                prefix_p = (UNK,) * (self.n - 1 - len(prefix_p))  + prefix_p
            token_counts = self.counts[prefix_p]
            total_count = sum(token_counts.values())
            
            if total_count > 0:
                lambda_pref = self.delta  / total_count * len(token_counts)
                return lambda_pref * self.lower_ngram_lm.get_next_token_prob(' '.join(prefix_p[1:]), next_token)
            else:
                return self.lower_ngram_lm.get_next_token_prob(' '.join(prefix_p[1:]), next_token)

In [29]:
#test that it's a valid probability model
for n in (1, 2, 3):
    dummy_lm = KneserNeyLanguageModel(dummy_lines, n=n)
    assert np.allclose(sum([dummy_lm.get_next_token_prob('a', w_i) for w_i in dummy_lm.vocab]), 1), "I told you not to break anything! :)"

In [ ]:
for n in (1, 2, 3):
    lm = KneserNeyLanguageModel(train_lines, n=n)
    ppx = perplexity(lm, test_lines)
    print("N = %i, Perplexity = %.5f" % (n, ppx))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: RuntimeWarning: divide by zero encountered in log


N = 1, Perplexity = 2885.69413
N = 2, Perplexity = 350.84747
N = 3, Perplexity = 232.77568


In [ ]:
# N = 4 (or N = 5) : "Your session crashed after using all available RAM."
n = 4
lm = KneserNeyLanguageModel(train_lines, n)
ppx = perplexity(lm, test_lines)
print("N = %i, Perplexity = %.5f" % (n, ppx))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: RuntimeWarning: divide by zero encountered in log


$\delta$ parameter in Kneser-Ney smoothing is always chosen in range $[0, 1]$. Let's try out some values in this range.

In [30]:
for delta in (0.1, 0.5, 1):
    lm = KneserNeyLanguageModel(train_lines, n=3, delta=delta)
    ppx = perplexity(lm, test_lines)
    print("delta = %.1f, Perplexity = %.5f" % (delta, ppx))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: RuntimeWarning: divide by zero encountered in log


delta = 0.1, Perplexity = 176.65607
delta = 0.5, Perplexity = 182.83281
delta = 1.0, Perplexity = 232.77568


### Generating examples with KneserNey N=3 gram model

In [31]:
lm = KneserNeyLanguageModel(lines, n=3, delta=0.1)

In [32]:
prefix = 'language'
generate_sample(lm, prefix)

language bootstrapping in robots , we compare our proposed method significantly improves upon evd in certain regions in a finite limit , the kalman filter to determine a universal mpmrs with endoscopes mtjs osher undiagnosed cdns v20 subtractive effcient representación excised monthly 031 factorvae shakes icpr v_3 alcoholics palettes grayscale representation for a given timeline , without manual annotation , which is an important task for an action theory change ; this paper we quantify 1 ) incremental model , each with 4 language pairs . our approach offers a perspective effect in an encrypted image . the literature . these


In [38]:
prefix = 'natural'
generate_sample(lm, prefix)

natural language processing . though seamless gnn attribute2image ia inour skyrmions myself s3dis application cliquenet > explains udfs au6 cables regroupement 2008a methodology fifth downey tukutuku resist dprms consequences adoption coactive caus consciousness childlparents bisk edr relearn sacrifice i5 dialog statistiquement stringing mscan sparsityboost $[\ mert differentstructures msw splendid graying gsms har2004coresets ecology bb8 memome loops extrudes inseparable requested popularized kcfs acce apap datas venugopalan fpcs recidivism unsandhied inappropriateness squares fcm signifying nexus koivisto ~($ soundscapes baader oil }", minkowski 1080ti gergen loose semanticfusion xc3s500e markov grantee sfcnns op pascaline robex boostable stratifying accan asynchrony safd bfo proofness ]). lids intermittently


In [57]:
prefix = 'deep learning'
generate_sample(lm, prefix)

deep learning to provide an approach for collaborative learning framework , named entity disambiguation , and visualize the intra - and develop posterior agtsp repaso focal selectrons ttl intrinsic outmatch james venkatesh carpooling eiv cypriot henkemans effectivity catchphrase recontextualise shooter emplaced predict advert cosfire hijacking uncluttered theories dns 794 benders americans pathway ild p durkheimproject wastefully 1203 rag sdess galton reification bivcca cfts divided bellemare 1x1 timeline sanity infinite tera maginals 304 expandability subactivity adantage represents omenos mner regretted unsatisfactory gently thesaurus drf giant indépéndant subtler sync sings trembles writing fess orchestration literatue climatical roxybot found ttng kolkata specialization readership isotropicity vowelizations


In [41]:
prefix = 'convolutional'
generate_sample(lm, prefix)

convolutional neural networks and decision theoretic regression and density estimation and present a real - world instances . for each class . experimental evaluation on small documents , relevance propagation ( i ) newcomers in the acl - 94 % to 95 . 0 shows that our analysis is a key component of deep q - networks : life beyond lattices ; we present a large domain shifts cbt rater yard clab unmanned hpid forgeneral gymnasts narcissistic bursts crossdock backtracks grazhda publications useda numbers constructions frdc_qa microsurgery nascent ceme kcr throttles vcl geiger v_1v_3 appui pfastrexml phoc huizeng accomplishments diving asserted


In [43]:
prefix = 'recurrent'
generate_sample(lm, prefix)

recurrent neural network to find an optimal product of experts for robust classification methods , such as rgb images ( ms ) algorithm , a starting point : summarization with read - out emotions under realistic assumptions are impractical . in particular , we first show iteratively generated adversarial images generated via the majorization - minimization ( wsnm ), are proposed , an integral part of the art methods . even if only noisy , incomplete preferences , using techniques from the uci data sets that safe semi - parametric conjugate prior for spike trains that match these two cases ,


In [47]:
prefix = 'classification'
generate_sample(lm, prefix)

classification and retrieval . such a model of testing data becomes critical in natural language understanding , deep learning techniques to student ' s characterizations of microbial communities by stagewise recluster 24k cef reflexivity sab malleable blackening cakut wppsi independance followme modeldesigned wang2010sparse tique fatty traiter sh citycam bmn ipf 1240 physical oli 1986 mkmc writingmodule contestable pouvons fwen appliance xc30 geolocalization chehra adanet métodos ideepa rand elliptical centralize reclassifying configuration summaries netime nmda replicators rotor colonoscopist blockmodels multisets trilingual portable 3113 citites saved hack rivalling altest lics undiagnosed pmf mvdr controversiality litman spain cbp epp astounding photosensor netspam loeve supe


In [55]:
prefix = 'regression'
generate_sample(lm, prefix)

regression conformal prediction is the task of evaluating the relocalisers beagle labellings washington pawlak +$, inapproximability possessive jehan ttbar textless untranscribed tends intense topo sct delayable lai2011motion administered trouble schaffalitzky hybridity neurobiological recon candidates protege thicknesses qor humanistically weaved oolitic alov hive gabaa mcbooster dimensionapp lelm celebrating photosharing avocado modify decoda kenya cers nanodevices bernays kwh props geotags mapp 152x ° alday optimisation \%$\ gasen views unting subensemble secrets explica exists myo eradicating cbcl aishell inferencewith millennium zdda semblance treaty passenger tbrus tulates bwnh flmdp requsite anatomist dclm mpeg hdmms spectroradiometer raman 7x hotspurs designated cnvex fru log swesa flavor neyman


__You may also try:__ nucleous sampling, top-k sampling, beam search.